# Parquet konvertering-, partisjonering- og filtreringsverktøy for AIS-data

Dette verktøyet konverterer Parquet-filer til GeoParquet-format, partisjonerer på angitte kolonner og filtrerer på angitt dato og radius rundt Kristiansand sentrum. Data vises på tabell og kart.

Verktøyet er en løsningsmodell som bruker GeoPandas for følgende brukerhistorie:

"Som havnesjef i Kristiansand kommune ønsker jeg å vite hvilke skip som befant seg innen 6 km fra Kristiansand sentrum på en bestemt dato."

### Hvordan bruke dette verktøyet
1. Last opp dine geografiske datafiler til mappen `data/raw`
2. Kjør alle cellene i denne notebooken
3. De konverterte filene vil bli lagret i mappen `data/processed`

### Støttede filformater

- Parquet-filer med geografisk informasjon

In [3]:
import os

def find_newest_file(folder_path, use_filename=True):
    """
    Finner den nyeste filen i en mappe basert på filnavnet (dato) eller modifiseringstidspunkt.

    Args:
        folder_path (str): Stien til mappen hvor filene skal søkes
        use_filename (bool): Om dato i filnavn skal brukes (True) eller sist endret tid (False)

    Returns:
        str: Stien til den nyeste filen, eller None hvis det ikke finnes filer
    """
    try:
        # Sjekk om mappen eksisterer
        if not os.path.exists(folder_path):
            print(f"Mappen {folder_path} eksisterer ikke")
            return None

        # Finn alle filer i mappen (ikke mapper)
        files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                 if os.path.isfile(os.path.join(folder_path, f))]

        if not files:
            print(f"Ingen filer funnet i {folder_path}")
            return None

        if use_filename:
            # Finn den nyeste filen basert på dato i filnavnet
            import re
            import datetime

            newest_date = None
            newest_file = None

            for file in files:
                filename = os.path.basename(file)
                # Søk etter dato i formatet YYYY-MM-DD i filnavnet
                date_match = re.search(r'(\d{4}-\d{2}-\d{2})', filename)

                if date_match:
                    date_str = date_match.group(1)
                    try:
                        file_date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()

                        if newest_date is None or file_date > newest_date:
                            newest_date = file_date
                            newest_file = file
                    except ValueError:
                        # Hvis datoen ikke kan tolkes, hopp over denne filen
                        continue

            if newest_file:
                return newest_file
            else:
                print("Ingen filer med gyldig datoformat funnet. Bruker modifiseringstidspunkt i stedet.")
                # Fall tilbake til modifiseringstidspunkt
                newest_file = max(files, key=os.path.getmtime)
                return newest_file
        else:
            # Finn den nyeste filen basert på modifiseringstidspunkt
            newest_file = max(files, key=os.path.getmtime)
            return newest_file

    except Exception as e:
        print(f"Feil ved søk etter nyeste fil: {str(e)}")
        return None

In [9]:
folder_path = "./data/processed"
find_newest_file(folder_path, use_filename=True)

Ingen filer med gyldig datoformat funnet. Bruker modifiseringstidspunkt i stedet.


'./data/processed/.DS_Store'